In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import requests

import warnings
warnings.filterwarnings('ignore')

In [17]:
top_10_snp_500_tickers = [
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "AMZN",  # Amazon.com Inc.
    "GOOGL",  # Alphabet Inc. (Class A)
    "GOOG",  # Alphabet Inc. (Class C)
    "META",  # Meta Platforms Inc.
    "TSLA",  # Tesla Inc.
    "JPM",  # JPMorgan Chase & Co.
    "NVDA",  # NVIDIA Corporation
    "UNH"  # UnitedHealth Group Incorporated
]
tickers = top_10_snp_500_tickers

In [21]:
# Download data for the last 20 calendar days to ensure we capture at least 11 market days
df = yf.download(tickers=tickers, period='1mo').stack()

# Clean up the columns and index
df = df.reset_index()  # Reset index to work with columns
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(' ', '')
df.set_index(['date', 'ticker'], inplace=True)  # Set the index back

# Filter to get the last 11 market days
df = df.groupby(level='ticker').tail(11)

df

[*********************100%%**********************]  10 of 10 completed


Price                adjclose       close        high         low        open  \
date       ticker                                                               
2024-07-01 AAPL    216.750000  216.750000  217.509995  211.919998  212.089996   
           AMZN    197.199997  197.199997  198.300003  192.820007  193.490005   
           GOOG    184.490005  184.490005  185.339996  182.729996  184.479996   
           GOOGL   182.990005  182.990005  183.880005  181.300003  183.029999   
           JPM     204.317856  205.449997  207.089996  202.660004  202.839996   
...                       ...         ...         ...         ...         ...   
2024-07-16 META    488.860107  488.860107  503.949890  485.790009  501.500000   
           MSFT    449.279999  449.279999  454.299988  446.660004  454.220001   
           NVDA    125.889900  125.889900  129.039993  124.580002  128.410004   
           TSLA    255.772797  255.772797  258.619995  245.800095  255.320007   
           UNH     548.179993  548.179993  548.209900  527.130005  533.900024   

Price                 volume  
date       ticker             
2024-07-01 AAPL     60402900  
           AMZN     41192000  
           GOOG     11815900  
           GOOGL    16006100  
           JPM      10205800  
...                      ...  
2024-07-16 META     11165335  
           MSFT     11804112  
           NVDA    174653437  
           TSLA    114026921  
           UNH       8718216  

[110 rows x 6 columns]